# Auto Generated Agent Chat: Using MathChat to Solve Math Problems

AutoGen offers conversable agents powered by LLM, tool or human, which can be used to perform tasks collectively via automated chat. This framework allows tool use and human participation through multi-agent conversation. Please find documentation about this feature [here](https://microsoft.github.io/autogen/docs/Use-Cases/agent_chat).

MathChat is an experimental convesational framework for math problem solving. In this notebook, we demonstrate how to use MathChat to solve math problems. MathChat uses the `AssistantAgent` and `MathUserProxyAgent`, which is similar to the usage of `AssistantAgent` and `UserProxyAgent` in other notebooks (e.g., [Automated Task Solving with Code Generation, Execution & Debugging](https://github.com/microsoft/autogen/blob/main/notebook/agentchat_auto_feedback_from_code_execution.ipynb)). Essentially, `MathUserProxyAgent` implements a different auto reply mechanism corresponding to the MathChat prompts. You can find more details in the paper [An Empirical Study on Challenging Math Problem Solving with GPT-4](https://arxiv.org/abs/2306.01337) or the [blogpost](https://microsoft.github.io/autogen/blog/2023/06/28/MathChat).

## Requirements

AutoGen requires `Python>=3.8`. To run this notebook example, please install the [mathchat] option.
```bash
pip install "pyautogen[mathchat]"
```

In [ ]:
# %pip install "pyautogen[mathchat]~=0.1.1"

## Set your API Endpoint

The [`config_list_from_json`](https://microsoft.github.io/autogen/docs/reference/oai/openai_utils#config_list_from_json) function loads a list of configurations from an environment variable or a json file.


In [ ]:
import autogen

config_list = autogen.config_list_from_json(
    "OAI_CONFIG_LIST",
    filter_dict={
        "model": {
            "gpt-4",
            # "gpt-3.5-turbo",
            # "text-davinci-003",
        }
    }
)

## Construct agents for MathChat

We start by initialzing the `AssistantAgent` and `MathUserProxyAgent`. The system message needs to be set to "You are a helpful assistant." for MathChat. The detailed instructions are given in the user message. Later we will use the `MathUserProxyAgent.generate_init_message` to combine the instructions and a math problem for an initial message to be sent to the LLM assistant.

In [ ]:
from autogen.agentchat.contrib.math_user_proxy_agent import MathUserProxyAgent

autogen.ChatCompletion.start_logging()

# 1. create an AssistantAgent instance named "assistant"
assistant = autogen.AssistantAgent(
    name="assistant", 
    system_message="You are a helpful assistant.",
    llm_config={
        "request_timeout": 600,
        "seed": 42,
        "config_list": config_list,
    }
)

# 2. create the MathUserProxyAgent instance named "mathproxyagent"
# By default, the human_input_mode is "NEVER", which means the agent will not ask for human input.
mathproxyagent = MathUserProxyAgent(
    name="mathproxyagent", 
    human_input_mode="NEVER",
    code_execution_config={"use_docker": False},
)

### Example 1

Problem: Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation.

Correct Solution: 
We have \begin{align*} (2x+10)(x+3)&<(3x+9)(x+8) \quad \Rightarrow
\\ 2(x+5)(x+3)&<3(x+3)(x+8) \quad \Rightarrow
\\ 2(x+5)(x+3)-3(x+3)(x+8)&<0 \quad \Rightarrow
\\ (2x+10-(3x+24))(x+3)&<0 \quad \Rightarrow
\\ (-x-14)(x+3)&<0 \quad \Rightarrow
\\ (x+14)(x+3)&>0.
\end{align*} This inequality is satisfied if and only if $(x+14)$ and $(x+3)$ are either both positive or both negative.  Both factors are positive for $x>-3$ and both factors are negative for $x<-14$.  When $-14<x<-3$, one factor is positive and the other negative, so their product is negative.   Therefore, the range of $x$ that satisfies the inequality is $ \boxed{(-\infty, -14)\cup(-3,\infty)} $.

In [ ]:
# given a math problem, we use the mathproxyagent to generate a prompt to be sent to the assistant as the initial message.
# the assistant receives the message and generates a response. The response will be sent back to the mathproxyagent for processing.
# The conversation continues until the termination condition is met, in MathChat, the termination condition is the detect of "\boxed{}" in the response.
math_problem = "Find all $x$ that satisfy the inequality $(2x+10)(x+3)<(3x+9)(x+8)$. Express your answer in interval notation."
mathproxyagent.initiate_chat(assistant, problem=math_problem)

### Example 2
Problem: For what negative value of $k$ is there exactly one solution to the system of equations \begin{align*}
y &= 2x^2 + kx + 6 \\
y &= -x + 4?
\end{align*}

Correct Solution: Setting the two expressions for $y$ equal to each other, it follows that $2x^2 + kx + 6 = -x + 4$. Re-arranging, $2x^2 + (k+1)x + 2 = 0$. For there to be exactly one solution for $x$, then the discriminant of the given quadratic must be equal to zero. Thus, $(k+1)^2 - 4 \cdot 2 \cdot 2 = (k+1)^2 - 16 = 0$, so $k+1 = \pm 4$. Taking the negative value, $k = \boxed{-5}$.

In [ ]:
math_problem = "For what negative value of $k$ is there exactly one solution to the system of equations \\begin{align*}\ny &= 2x^2 + kx + 6 \\\\\ny &= -x + 4?\n\\end{align*}"
mathproxyagent.initiate_chat(assistant, problem=math_problem)